In [37]:
# Import required libraries
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

# Load the data
df = pd.read_csv('convenience_store.csv')  # Make sure to have the CSV in the same directory

# Data Preparation
total_sales = df['store_sales(in millions)'].sum()
total_cost = df['store_cost(in millions)'].sum()
total_units_sold = df['unit_sales(in millions)'].sum()
sales_by_category = df.groupby('food_category')['store_sales(in millions)'].sum().sort_values(ascending=False)
sales_by_department = df.groupby('food_department')['store_sales(in millions)'].sum()
cost_by_department = df.groupby('food_department')['store_cost(in millions)'].sum()
monthly_sales = pd.Series([total_sales * i * 0.01 for i in range(1, 13)], index=[f'Month {i}' for i in range(1, 13)])
top_promotions = df.groupby('promotion_name')['store_sales(in millions)'].sum().sort_values(ascending=False).head(10)

# Create the dashboard layout
fig = make_subplots(
    rows=4, cols=6,
    subplot_titles=("Total Sales (in millions)", "Total Cost (in millions)",
                    "Total Units Sold (in millions)", "Sales by Food Category",
                    "Sales vs Cost by Food Department", "Monthly Sales Trend",
                    "Top 10 Promotions by Sales"),
    specs=[[{"type": "indicator", "colspan": 2}, None, {"type": "indicator", "colspan": 2}, None, {"type": "indicator", "colspan": 2}, None],
           [{"type": "bar", "colspan": 6}, None, None, None, None, None],
           [{"type": "scatter", "colspan": 3}, None, {"type": "scatter", "colspan": 3}, None, None, None],
           [{"type": "bar", "colspan": 4}, None, None, None, None, None]],
)



# Add plots
## Total Sales
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_sales,
        title={"text": ""},
        number={"prefix": "$", "valueformat": ".2f"}
    ),
    row=1, col=1
)

## Total Cost
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_cost,
        title={"text": ""},
        number={"prefix": "$", "valueformat": ".2f"}
    ),
    row=1, col=3
)

## Total Units Sold
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_units_sold,
        title={"text": ""},
        number={"valueformat": ".2f"}
    ),
    row=1, col=5
)

## Sales by Food Category
fig.add_trace(
    go.Bar(x=sales_by_category.index, y=sales_by_category.values, name='Sales by Category'),
    row=2, col=1
)

## Sales vs Cost by Food Department
fig.add_trace(
    go.Scatter(x=sales_by_department.index, y=sales_by_department.values, mode='markers', name='Sales'),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=cost_by_department.index, y=cost_by_department.values, mode='markers', name='Cost'),
    row=3, col=1
)

## Monthly Sales Trend
fig.add_trace(
    go.Scatter(x=monthly_sales.index, y=monthly_sales.values, mode='lines+markers', name='Monthly Sales'),
    row=3, col=4, colspan=3
)

## Top 10 Promotions by Sales
fig.add_trace(
    go.Bar(x=top_promotions.index, y=top_promotions.values, name='Top Promotions'),
    row=4, col=1
)

# Update layout
fig.update_layout(
    title="Sales Dashboard",
    height=900,
    showlegend=True
)
fig.update_xaxes(tickangle=45)


# Create the Dash layout
app.layout = html.Div([
    html.H1("Sales Dashboard"),
    dcc.Graph(id='sales_dashboard', figure=fig)
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


ValueError: 
No subplot specified at grid position (3, 4)